In [1]:
import numpy as np
import tensorflow as tf
from models.official.mnist import dataset
tf.enable_eager_execution()
#tf.logging.set_verbosity(tf.logging.INFO)

/opt/conda/envs/riptide-py3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [57]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    #input_layer = tf.reshape(features, [-1, 28, 28, 1])
    tf.summary.image("Inputs", features)
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=features,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    #accuracy = tf.metrics.accuracy(
    #    labels=labels, predictions=tf.argmax(logits, axis=1))
    # can use this for more logging
    #tf.identity('cross_entropy', loss)
    #tf.summary.scalar('train_accuracy', accuracy[1])
    tf.summary.scalar("loss", loss)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        # can use name scope in layer creation then 
        #"scope/prefix/for/first/vars")
        # first_train_op = optimizer.minimize(cost, var_list=first_train_vars)
        # can combine multiple train ops with tf.group(op1, op2)
        # print the sum of losses and use tensorboard to visualize each
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [2]:
def _preprocess(features, label):
    features = tf.reshape(features, [-1, 28, 28, 1])
    return features, label

def train_input_fn():
    ds = dataset.train("/tmp")
    ds = ds.cache().shuffle(buffer_size=50000).batch(100)
    ds = ds.map(_preprocess)
    ds = ds.prefetch(tf.contrib.data.AUTOTUNE)
    ds = ds.repeat()
    return ds

def eval_input_fn():
    ds = dataset.test("/tmp")
    ds = ds.cache().shuffle(buffer_size=50000).batch(100)
    ds = ds.map(_preprocess)
    return ds

In [3]:
ds = eval_input_fn()

In [5]:
iter(ds).next()[1]

<tf.Tensor: id=56, shape=(100,), dtype=int32, numpy=
array([2, 6, 3, 4, 7, 3, 1, 0, 3, 3, 0, 8, 2, 2, 3, 1, 2, 8, 5, 7, 7, 4,
       3, 0, 3, 8, 8, 6, 8, 3, 2, 4, 5, 5, 8, 4, 5, 5, 9, 7, 0, 1, 7, 4,
       7, 3, 1, 2, 7, 1, 6, 9, 1, 3, 8, 7, 4, 2, 8, 1, 8, 6, 3, 4, 3, 4,
       0, 8, 8, 6, 0, 9, 7, 1, 3, 3, 1, 8, 5, 2, 2, 1, 3, 3, 4, 2, 2, 5,
       6, 3, 7, 6, 1, 1, 8, 9, 9, 0, 0, 0], dtype=int32)>

In [59]:
# Create the Estimator
strategy = tf.contrib.distribute.MirroredStrategy(num_gpus=2)
run_config = tf.estimator.RunConfig(train_distribute=strategy)
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/data/jwfromm/models/mnist_convnet_model", config=run_config)

INFO:tensorflow:Using config: {'_model_dir': '/data/jwfromm/models/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.contrib.distribute.python.mirrored_strategy.MirroredStrategy object at 0x7efc7c9096d8>, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efc7c909898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [60]:
# Set up logging for predictions
#tensors_to_log = {"Loss": "cross_entropy", "Train Acc": "train_accuracy"}
#logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

In [61]:
mnist_classifier.train(input_fn=train_input_fn, steps=20000)

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_CPU:0
INFO:tensorflow:Device is available but not used by distribute strategy: /device:XLA_GPU:0
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /data/jwfromm/models/mnist_convnet_model/model.ckpt-40004
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 40005 into /data/jwfromm/models/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 0.07438053, step = 40004
INFO:tensorflow:global_step/sec: 88.5354
INFO:tensorflow:loss = 0.049069062, step = 40104 (1.131 sec)
INFO:tensorflow:global_step/sec: 174.005
INFO:tensorflow:loss = 0.07

KeyboardInterrupt: 

In [7]:
mnist_classifier.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-09-05:47:03
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /data/jwfromm/models/mnist_convnet_model/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-09-05:47:04
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9686, global_step = 20000, loss = 0.10597835


{'accuracy': 0.9686, 'loss': 0.10597835, 'global_step': 20000}